In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

In [5]:
spark = SparkSession.builder \
.appName("datascientistsalaries_dataset") \
.getOrCreate()

In [6]:
df = spark.read \
.option("header",True) \
.option("inferSchema","True") \
.csv("/Users/burak/Desktop/Software Projects/Data Engineering/Data Scientist Salaries/ds_salaries.csv")

In [7]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



In [8]:
#Sort by salary_in_usd in descending order
df.orderBy(F.desc("salary_in_usd")).select("work_year",\
"experience_level","job_title","salary_in_usd",\
"company_location","company_size").show(10)

+---------+----------------+--------------------+-------------+----------------+------------+
|work_year|experience_level|           job_title|salary_in_usd|company_location|company_size|
+---------+----------------+--------------------+-------------+----------------+------------+
|     2021|              EX|Principal Data En...|       600000|              US|           L|
|     2021|              MI|Financial Data An...|       450000|              US|           L|
|     2020|              MI|  Research Scientist|       450000|              US|           M|
|     2021|              MI|Applied Machine L...|       423000|              US|           L|
|     2021|              EX|Principal Data Sc...|       416000|              US|           S|
|     2020|              SE|      Data Scientist|       412000|              US|           L|
|     2022|              SE| Data Analytics Lead|       405000|              US|           L|
|     2022|              SE|Applied Data Scie...|       3800

In [9]:
#count of job_titles
df.select(F.countDistinct("job_title")).alias("Count Of Job Titles").show()

+-------------------------+
|count(DISTINCT job_title)|
+-------------------------+
|                       50|
+-------------------------+



In [10]:
#number of job titles with their types
df.groupBy("job_title").count().orderBy(F.desc("count")).select(col("job_title").alias("Job Title"),col("count").alias("Count")).show(10)


+--------------------+-----+
|           Job Title|Count|
+--------------------+-----+
|      Data Scientist|  143|
|       Data Engineer|  132|
|        Data Analyst|   97|
|Machine Learning ...|   41|
|  Research Scientist|   16|
|Data Science Manager|   12|
|      Data Architect|   11|
|Machine Learning ...|    8|
|   Big Data Engineer|    8|
|Director of Data ...|    7|
+--------------------+-----+
only showing top 10 rows



In [11]:
#longest job titles
df.select(col("job_title").alias("Job Title"),length("job_title").alias("Long Job Title")).orderBy(F.desc("job_title")).\
limit(1).show(truncate=False)

+--------------------+--------------+
|Job Title           |Long Job Title|
+--------------------+--------------+
|Staff Data Scientist|20            |
+--------------------+--------------+



In [14]:
#100% number of remote workers in Germany and sort salary
df.where((col("company_location")=="DE")&(col("remote_ratio")==100)).\
orderBy("salary").select("experience_level","job_title","salary","remote_ratio","company_location").\
show()

+----------------+--------------------+------+------------+----------------+
|experience_level|           job_title|salary|remote_ratio|company_location|
+----------------+--------------------+------+------------+----------------+
|              EN|         ML Engineer| 14000|         100|              DE|
|              MI|      Data Scientist| 21600|         100|              DE|
|              EN|       Data Engineer| 48000|         100|              DE|
|              MI|Data Engineering ...| 51999|         100|              DE|
|              EN|       Data Engineer| 52800|         100|              DE|
|              EN|Data Science Cons...| 65000|         100|              DE|
|              SE|Data Analytics En...| 67000|         100|              DE|
|              MI|Machine Learning ...| 80000|         100|              DE|
|              EN|Machine Learning ...| 85000|         100|              DE|
|              SE|Principal Data Sc...|130000|         100|              DE|

In [ ]:
#